# Import Packages

In [107]:
import requests
import time
import ollama
from openai import OpenAI
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import Markdown, display

import credentials

# Data Scraping

In [73]:
USERNAME = credentials.credentials.username
PASSWORD = credentials.credentials.password

LINKEDIN_LOGIN = "https://www.linkedin.com/login"

profile_urls = ["https://www.linkedin.com/in/yusup-winata/", "https://www.linkedin.com/in/firlyana-9099a3265/"]

In [78]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = Chrome(options=chrome_options)

profile_names = []
profile_summaries = []

# Login
print("Sign-in to LinkedIn")
driver.get(LINKEDIN_LOGIN)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(USERNAME)
driver.find_element(By.ID, "password").send_keys(PASSWORD)
driver.find_element(By.XPATH, "//button[@type='submit']").click()
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "title")))
print(f"Successfully login to LinkedIn")

# Scrap profile summary
for url in profile_urls:
    print(f"\nScraping {url}")
    driver.get(url)

    profile_xpath = "//h1[contains(@class, 'inline') and contains(@class, 'v-align-middle')]"
    profile_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, profile_xpath)))
    print(f"Name: {profile_name.text}")
    profile_names.append(profile_name.text)
    
    summary_xpath = "//div[contains(@class, 'display-flex') and contains(@class, 'ph5')]//span[@aria-hidden='true']"
    profile_summary = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, summary_xpath)))
    print(f"Summary: {profile_summary.text}")
    profile_summaries.append(profile_summary.text)
    
driver.quit()

Sign-in to LinkedIn
Successfully login to LinkedIn

Scraping https://www.linkedin.com/in/yusup-winata/
Name: Yusup Hidayat Winata
Summary: A passionate Artificial Intelligence & Machine Learning Enthusiast with more than 2 years of experience in AI and ML development, mentorship, and program management. Skilled in building and deploying AI models, conducting research, and developing user-friendly applications. Proficient in various ML techniques, including classification, regression, clustering, NLP, and Computer Vision, with a strong foundation in Deep Learning and Python. Committed to continuous learning and leveraging AI to solve real-world problems.

Scraping https://www.linkedin.com/in/firlyana-9099a3265/
Name: Firlyana .
Summary: I have experience working in insurance companies and insurance brokers


# Prepare Model and Prompt

**Note:** Need to install ollama and llama3.2 first.

In [82]:
!ollama list

NAME                ID              SIZE      MODIFIED    
llama3.2:latest     a80c4f17acd5    2.0 GB    2 hours ago    
deepseek-r1:1.5b    a42b25d8c10a    1.1 GB    3 hours ago    
tinyllama:latest    2644915ede35    637 MB    4 hours ago    


In [85]:
!ollama pull llama3.2

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest 
pulling dde5aa3fc5ff... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 2.0 GB                         
pulling 966de95ca8a6... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.4 KB                         
pulling fcc5a6bec9da... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 7.7 KB                         
pulling a70ff7e570d9... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 6.0 KB                         
pulling 56bb8bd477a5... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�   96 B                         
pulling 34bb5ab01051... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [108]:
BASE_URL = "http://localhost:11434/v1"
API_KEY = "ollama"
MODEL = "llama3.2"

ollama_openai = OpenAI(api_key=API_KEY, base_url=BASE_URL)

system_prompt = "You are an assistant that analyzes the summary of a LinkedIn profile and provides constructive feedbacks. Respond in markdown."

def create_user_prompt(profile_name, profile_summary, system_prompt=system_prompt):
    user_prompt = system_prompt
    user_prompt += f"\n\nNow you are looking at {profile_name}'s LinkedIn profile summary."
    user_prompt += "\nThe summary as follows; please provide constructive feedbacks for the summary.\n\n"
    user_prompt += profile_summary
    return user_prompt

def create_messages(profile_name, profile_summary):
    return [{"role": "user",
             "content": create_user_prompt(profile_name, profile_summary)}]

def get_feedback(profile_name, profile_summary, model=MODEL):
    feedback = ollama_openai.chat.completions.create(model=model, messages=create_messages(profile_name, profile_summary))
    return feedback.choices[0].message.content

def display_feedback(profile_name, profile_summary):
    feedback = get_feedback(profile_name, profile_summary)
    display(Markdown(feedback))

# Prompting

In [116]:
for profile_name, profile_summary in zip(profile_names, profile_summaries):
    display_feedback(profile_name, profile_summary)
    print(100*"=")

**Summary Feedback for Yusup Hidayat Winata's LinkedIn Profile**

### Strengths:

*   The summary clearly states your passion for Artificial Intelligence (AI) and Machine Learning (ML), which is a great starting point.
*   You've highlighted your experience in AI and ML development, mentorship, and program management.
*   Your proficiency in various ML techniques and Python skills are well-represented.

### Suggestions:

*   **Make it more specific**: While "building and deploying AI models" and "developing user-friendly applications" are mentioned, consider adding more concrete examples or projects that showcase your practical experience.
*   **Use action verbs consistently**: Use active voice throughout the summary to create a more engaging tone.
*   **Quantify accomplishments**: Adding metrics (e.g., "improved model accuracy by \[X%]") can demonstrate the impact of your work and make it more interesting.
*   **Emphasize soft skills**: In addition to technical expertise, highlight any relevant collaboration or project management experience that can benefit future employers.
*   **Professional tone and clarity**: Reedit for clarity and concise statements.

### Example Revamped Summary

"I'm a passionate AI & Machine Learning enthusiast with  [number] years of experience driving solutions. I've successfully developed and deployed multiple AI models, improving user engagement by \[X%]. 

Proficient in key techniques like Deep Learning, classification, regression, clustering, NLP, and computer vision using Python, backed by strong foundational knowledge."

**Summary Feedback**
===============

**Overall Assessment**
--------------------

Firlyana's summary is brief, but it could be more informative and engaging. The current statement only mentions her work experience, which doesn't give a clear idea of her value proposition or what sets her apart from others in the industry.

**Suggestions for Improvement**
-----------------------------

* Instead of just listing her job roles (insurance companies and insurance brokers), provide specific examples or accomplishments that demonstrate her expertise in this field.
* Consider adding a brief description of how she contributes to organizations, such as "results-driven professional with a proven track record of driving business growth" or "strategic partner with excellent communication skills."
* Think about highlighting any notable certifications, awards, or recognition Firlyana has received, as these can add credibility to her summary.
* Use action verbs like "managed," "created," "developed," or "improved" to describe her experiences and achievements.

**Revised Summary Example**
---------------------------

"As a seasoned insurance professional with 5+ years of experience in the industry, I've developed a deep understanding of the market and a proven track record of driving business growth through strategic partnerships and innovative solutions. Most recently, I've served as a key account manager at XYZ Insurance Company, where I successfully managed a portfolio of high-profile clients and expanded the company's reach into new markets."